In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/gdrive')
# the project's folder
%cd /gdrive/'My Drive'/DataCuration
%ls

df = pd.read_csv("311Requests2021.csv")


#Get data from other files to populate missing values based on location boundaries

policeBeats = pd.read_csv("PoliceBeatDec2012.csv")
communityArea = pd.read_csv("CommAreas.csv")
wards = pd.read_csv("WARDS_2015.csv")


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/DataCuration
311Requests2021.csv  CommAreas.csv  PoliceBeatDec2012.csv  WARDS_2015.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install geopandas

In [ ]:
!pip install geopy
!pip install utm

In [ ]:
#Import geopy packages

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
locator = Nominatim()

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [ ]:

df = df[df['STREET_ADDRESS'].notna()] #Drop rows with empty street address
df = df[df['STREET_ADDRESS'].str.split(' ').str[-1].str.len() >= 2] #Rows rows with less than 4 words split
df['VALID_ADD'] = "VALID"
df['CITY'] = "CHICAGO" #Assign all city values to Chicago
df['STATE'] = "ILLINOIS" #Assign all city values to Illinois

COMMUNITY_AREA_nan = df[df['COMMUNITY_AREA'].isna()]
COMMUNITY_AREA_nan['ERROR'] = 'Missing community area'
print(COMMUNITY_AREA_nan.shape[0])

for ind1,sr in df.iterrows():
  splitarr = sr['STREET_ADDRESS'].split(" ")
  if(splitarr[-1] == "ST" and splitarr[2].isdigit()):
    #print(splitarr[2][-1])
    if len(splitarr)>4:
      splitarr.pop(3)
    if splitarr[2][-1] == '1':
      splitarr[2] = splitarr[2]+"st"
    elif splitarr[2][-1] == '2':
      splitarr[2] = splitarr[2]+"nd"
    elif splitarr[2][-1] == '3':
      splitarr[2] = splitarr[2]+"rd"
    else:
      splitarr[2] = splitarr[2]+"th"
    df.loc[ind1,['STREET_ADDRESS']] = ' '.join(splitarr)
  

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
#Check if address is valid and tag it
for ind1,sr in df.iterrows():
  address = sr['STREET_ADDRESS']+","+sr['CITY']+","+sr['STATE']
  if pd.isna(sr['ZIP_CODE']):
    try:
      location = locator.geocode(address, timeout=8, exactly_one=False)
      if location is None:
        df.loc[ind1,['VALID_ADD']] = "INVALID"
        continue;
      sr['ZIP_CODE'] = str(location[0]).split(',')[-2]
      df.loc[ind1,['ZIP_CODE']] = sr['ZIP_CODE']
      #print(str(location[0]).split(',')[-1])
      if(pd.isna(sr['LATITUDE']) or pd.isna(sr['LONGITUDE'])):
        df.loc[ind1,['LATITUDE']] = location[0].latitude
        df.loc[ind1,['LONGITUDE']] = location[0].longitude
        #print("lat_long:",ind1)
    
    except GeocoderTimedOut as e:
      df.loc[ind1,['VALID_ADD']] = "INVALID"
      print("Error: geocode failed on input %s with message "%(sr['STREET_ADDRESS']))
 
df[df['VALID_ADD'] == "INVALID"].shape[0]
df = df[df['VALID_ADD'] != "INVALID"]


,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,LAST_MODIFIED_DATE,CLOSED_DATE,STREET_ADDRESS,CITY,...,SANITATION_DIVISION_DAYS,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,X_COORDINATE,Y_COORDINATE,LATITUDE,LONGITUDE,LOCATION,VALID_ADD
0,SR22-00348638,Aircraft Noise Complaint,AVN,Aviation,Completed,03/03/2022 12:51:56 PM,03/03/2022 01:31:30 PM,03/03/2022 12:51:56 PM,10510 W ZEMKE RD,CHICAGO,...,NaN,12,5,3,1.105376e+06,1.941043e+06,41.994897,-87.887752,"(41.9948965498383, -87.88775165419361)",VALID
1,SR22-00348813,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,03/03/2022 01:10:55 PM,03/03/2022 01:32:04 PM,03/03/2022 01:10:55 PM,2111 W Lexington ST,CHICAGO,...,NaN,13,5,3,1.162024e+06,1.896642e+06,41.872060,-87.680615,"(41.872060463420134, -87.68061527067083)",VALID
2,SR22-00033271,No Water Complaint,WBJ,DWM - Department of Water Management,Canceled,01/07/2022 09:19:54 AM,01/07/2022 09:24:20 AM,01/07/2022 09:24:20 AM,5201 S MERRIMAC AVE,CHICAGO,...,NaN,9,6,1,1.135587e+06,1.869443e+06,41.797933,-87.778323,"(41.7979333264888, -87.77832289587248)",VALID
3,SR22-00033294,Graffiti Removal Request,GRAF,Streets and Sanitation,Completed,01/07/2022 09:21:51 AM,01/07/2022 09:21:58 AM,01/07/2022 09:21:58 AM,329 S FRANKLIN ST,CHICAGO,...,NaN,9,6,1,1.174496e+06,1.898621e+06,41.877221,-87.634766,"(41.8772206265366, -87.63476617277266)",VALID
4,SR22-00032999,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,01/07/2022 08:47:13 AM,01/07/2022 09:30:46 AM,01/07/2022 08:47:14 AM,2111 W Lexington ST,CHICAGO,...,NaN,8,6,1,1.162024e+06,1.896642e+06,41.872060,-87.680615,"(41.872060463420134, -87.68061527067083)",VALID


In [ ]:
import utm
import math
x,y,zone,dir = utm.from_latlon(41.728913, -87.632880)
print(x,y)

447364.38908533275 4619871.956555315


In [ ]:
df.loc[df['LATITUDE'].isnull(),'value_is_NaN'] = 'Yes'
df = df[df['value_is_NaN'] != "Yes"]
print(df.shape[0])
df['LOCATION'] = df[['LATITUDE', 'LONGITUDE']].apply(tuple, axis=1)
df['X_COORDINATE','Y_COORDINATE'] = df['LOCATION'].apply(lambda loc: utm.from_latlon(tuple(loc)[0],tuple(loc)[1]) if loc else None)


1242124


In [ ]:
df[['X_COORDINATE','Y_COORDINATE','temp','temp2']] = pd.DataFrame(df['X_COORDINATE','Y_COORDINATE'].tolist(), index=df.index)


In [ ]:
#Drop unnecessary columns
del df['VALID_ADD'],df['value_is_NaN'],df['temp'],df['temp2'],df['SANITATION_DIVISION_DAYS'],df['X_COORDINATE','Y_COORDINATE'],df['ELECTRICAL_DISTRICT'],df['ELECTRICITY_GRID']

In [ ]:
from shapely.geometry import shape
import math
import geopandas
from shapely.geometry import Polygon, LineString, Point,mapping
import shapely.wkt
import numpy as np


for ind1,sr in df.iterrows():
  #Get missing police information from CommAreas data
  if(math.isnan(sr['COMMUNITY_AREA'])):
    for ind3,commA in communityArea.iterrows():
      P = shapely.wkt.loads(commA['the_geom'])
      pt = Point(round(sr['LONGITUDE'],1),round(sr['LATITUDE'],1))
      s = geopandas.GeoSeries([P])
      if(pt.within(shape(P))):
          df.loc[ind1,['COMMUNITY_AREA']] = commA['AREA_NUM_1']
        
  
 #Get missing police information from policeBeat data
  if(math.isnan(sr['POLICE_BEAT'])):
    for ind2,rec in policeBeats.iterrows():
      P = shapely.wkt.loads(rec['the_geom'])
      pt = Point(round(sr['LONGITUDE'],1),round(sr['LATITUDE'],1))
      s = geopandas.GeoSeries([P])
      if(pt.within(P)):
        df.loc[ind1,['POLICE_DISTRICT']] = rec['DISTRICT']
        df.loc[ind1,['POLICE_SECTOR']] = rec['SECTOR']
        df.loc[ind1,['POLICE_BEAT']] = rec['BEAT_NUM']


  #Get missing ward information from wards2015 data
  if(math.isnan(sr['WARD'])):
    for ind4,ward in wards.iterrows():
      P = shapely.wkt.loads(ward['the_geom'])
      pt = Point(round(sr['LONGITUDE'],1),round(sr['LATITUDE'],1))
      s = geopandas.GeoSeries([P])
      if(pt.within(P)):
        df.loc[ind1,['WARD']] = ward['WARD']



In [ ]:
df.to_csv('output.csv')  

# VALIDATIONS


In [ ]:
COMMUNITY_AREA_nan = df[df['COMMUNITY_AREA'].isna()]
COMMUNITY_AREA_nan['ERROR'] = 'Missing community area'
print("No of null community: " ,COMMUNITY_AREA_nan.shape[0])
WARD_nan = df[df['WARD'].isna()]
WARD_nan['ERROR'] = 'Missing ward data'
print("No of null wards: " ,WARD_nan.shape[0])
POLICE_SECTOR_nan = df[df['POLICE_SECTOR'].isna()]
POLICE_SECTOR_nan['ERROR'] = 'Missing police data'
print("No of null police sectors: " ,POLICE_SECTOR_nan.shape[0]) 
POLICE_BEAT_nan = df[df['POLICE_BEAT'].isna()]
POLICE_BEAT_nan['ERROR'] = 'Missing police beat data'
print("No of null police beats: " ,POLICE_BEAT_nan.shape[0]) 
invalid_df = pd.concat([COMMUNITY_AREA_nan, WARD_nan, POLICE_SECTOR_nan, POLICE_BEAT_nan])
invalid_df.to_csv('validationReport.csv') 

No of null community:  181
No of null wards:  144
No of null police sectors:  151
No of null police beats:  151


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc